In [15]:
from pyspark.sql import SparkSession

DIRPATH = "C:/Users/sanjay s risbud/Dropbox/My Work/practice/python/pyspark/Spark-Tutorials-master/data/"
spark = SparkSession.builder.appName("Exercise").getOrCreate()

In [16]:
df = spark.read.csv(DIRPATH + "walmart_stock.csv", header=True, inferSchema=True)

In [17]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [18]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [19]:
for row in df.head(5):
    print(row, end="\n\n")

Row(Date='2012-01-03', Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996)

Row(Date='2012-01-04', Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475)

Row(Date='2012-01-05', Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539)

Row(Date='2012-01-06', Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922)

Row(Date='2012-01-09', Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)



In [20]:
df.describe().show()

+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|      Date|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|      1258|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean|      null| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|      null|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|2012-01-03|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|2016-12-30|         90.800003|        90.970001|            89.25|        90.4700

In [40]:
from pyspark.sql.functions import format_number
stats = df.describe()
columns = []
for name in stats.columns: 
    if name not in ["summary", "Date", "Volume"]:
        column = format_number(stats[name].cast("float"), 2).alias(name)
    elif name == "Volume":
        column = format_number(stats[name].cast("int"), 0).alias(name)
    else:
        column = name
    columns.append(column)
stats.select(columns).show()

+-------+----------+--------+--------+--------+--------+----------+---------+
|summary|      Date|    Open|    High|     Low|   Close|    Volume|Adj Close|
+-------+----------+--------+--------+--------+--------+----------+---------+
|  count|      1258|1,258.00|1,258.00|1,258.00|1,258.00|     1,258| 1,258.00|
|   mean|      null|   72.36|   72.84|   71.92|   72.39| 8,222,093|    67.24|
| stddev|      null|    6.77|    6.77|    6.74|    6.76| 4,519,780|     6.72|
|    min|2012-01-03|   56.39|   57.06|   56.30|   56.42| 2,094,900|    50.36|
|    max|2016-12-30|   90.80|   90.97|   89.25|   90.47|80,898,100|    84.91|
+-------+----------+--------+--------+--------+--------+----------+---------+



In [44]:
df.withColumn("HV Ratio", df["High"] / df["Volume"]).select(["Date", "HV Ratio"]).show()

+----------+--------------------+
|      Date|            HV Ratio|
+----------+--------------------+
|2012-01-03|4.819714653321546E-6|
|2012-01-04|6.290848613094555E-6|
|2012-01-05|4.669412994783916E-6|
|2012-01-06|7.367338463826307E-6|
|2012-01-09|8.915604778943901E-6|
|2012-01-10|8.644477436914568E-6|
|2012-01-11|9.351828421515645E-6|
|2012-01-12| 8.29141562102703E-6|
|2012-01-13|7.712212102001476E-6|
|2012-01-17|7.071764823529412E-6|
|2012-01-18|1.015495466386981E-5|
|2012-01-19|6.576354146362592...|
|2012-01-20| 5.90145296180676E-6|
|2012-01-23|8.547679455011844E-6|
|2012-01-24|8.420709512685392E-6|
|2012-01-25|1.041448341728929...|
|2012-01-26|8.316075414862431E-6|
|2012-01-27|9.721183814992126E-6|
|2012-01-30|8.029436027707578E-6|
|2012-01-31|6.307432259386365E-6|
+----------+--------------------+
only showing top 20 rows



In [51]:
df.filter(df["High"] == df.agg({"High": "max"}).first()[0]).collect()[0][0]

'2015-01-13'

In [53]:
df.agg({"Close": "avg"}).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



In [55]:
from pyspark.sql.functions import min, max
df.select(max(df["Volume"]), min(df["Volume"])).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



In [58]:
df.filter(df["Close"] < 60).count()

81

In [61]:
100 * (df.filter(df["High"] > 80).count())/df.count()

9.141494435612083

In [64]:
df.corr("High", "Volume")

-0.3384326061737161

In [70]:
df = df.withColumn("Year", df["Date"][0:4])
df.groupBy(df["Year"]).agg({"High": "max"}).orderBy("Year").show()

+----+---------+
|Year|max(High)|
+----+---------+
|2012|77.599998|
|2013|81.370003|
|2014|88.089996|
|2015|90.970001|
|2016|75.190002|
+----+---------+



In [72]:
df = df.withColumn("Month", df["Date"][6:2])
df.groupBy(df["Month"]).agg({"Close": "avg"}).orderBy("Month").show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|   01|71.44801958415842|
|   02|  71.306804443299|
|   03|71.77794377570092|
|   04|72.97361900952382|
|   05|72.30971688679247|
|   06| 72.4953774245283|
|   07|74.43971943925233|
|   08|73.02981855454546|
|   09|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+

